# Chains

- Prepare a Chat Model

In [32]:
import warnings
warnings.filterwarnings("ignore")

In [33]:
# =================== Section to change according to your choice of APIs you have access to===============
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint

repo_id = "mistralai/Mistral-7B-Instruct-v0.3"

# llm
llm = HuggingFaceEndpoint(
    repo_id=repo_id,
    task="text-generation",
    temperature=0.5,
    max_new_tokens=128  # Adjust max_new_tokens for generation
)

# Set up ChatHuggingFace
chatllm = ChatHuggingFace(llm=llm)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


##  LangChain Expression Language (LCEL) - which is LangChain's declarative way to compose chains using the pipe (|) (previously : LLMChain)

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["product"],
    template="What are 5 cool names for a {product}?"
)

chain = prompt | chatllm
result = chain.invoke({"product": "AI-powered toothbrush"})

In [13]:
print(result.content)

1. BrushBot: A smart, AI-powered toothbrush that uses advanced technology to ensure optimal oral hygiene.

2. SmartSweep: An AI toothbrush that adapts to your brushing habits, providing real-time feedback and guidance for a thorough clean.

3. PolarPerio: A futuristic, AI-driven toothbrush that monitors your dental health and provides personalized care recommendations.

4. OralOracle: An intelligent toothbrush that provides predictions and preventive care for your oral health with the help of its AI brain.

5. GumGuardian: An AI-powered toothbrush designed to protect and improve gum health, using smart sensors to monitor and adjust brushing patterns accordingly.


## Sequential Chains


- Simple Sequential Chain (single input/output)

In [16]:
from langchain.chains import SimpleSequentialChain, LLMChain

first_prompt = PromptTemplate(
    input_variables=["topic"],
    template="Generate a short story idea about {topic}."
)
first_chain = LLMChain(llm=llm, prompt=first_prompt)

second_prompt = PromptTemplate(
    input_variables=["story_idea"],
    template="Create a title for this story: {story_idea}"
)
second_chain = LLMChain(llm=llm, prompt=second_prompt)

simple_sequential_chain = SimpleSequentialChain(
    chains=[first_chain, second_chain],
    verbose=True
)
result = simple_sequential_chain.invoke("space exploration")



> Entering new SimpleSequentialChain chain...


Title: "Stardust Odyssey: The Last Frontier"

In the year 2135, humanity has reached the pinnacle of space exploration, with colonies on Mars and the Moon, and regular missions to the asteroid belt. However, a mysterious signal is detected emanating from a distant star system, sparking a new era of exploration.

Dr. Amelia Hart, a brilliant astrophysicist, decodes the signal and discovers it's an invitation from an alien civilization. The message reveals that the aliens have a powerful technology that could solve
 Earth's most pressing problems, but they're located in a remote and dangerous region of space known as the Last Frontier.

The United Earth Government (UEG) assembles an elite team of astronauts, scientists, and military personnel, led by Captain John Reynolds, to embark on a mission to the Last Frontier. The team includes Dr. Hart, who's been chosen for her expertise in decoding the alien language, and Lt. Commander James "Su

In [21]:
# With LCEL
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import  StrOutputParser


template1 = ChatPromptTemplate.from_template("Write a story about {topic}")
template2 = ChatPromptTemplate.from_template("Create a title for this story: {story}")

chain1 = template1 | llm | StrOutputParser()
chain2 = template2 | llm | StrOutputParser()

final_chain = {
    "story": chain1,
    "topic": RunnablePassthrough()
} | chain2

result = final_chain.invoke("a brave astronaut")

- Sequential Chain (multiple inputs/outputs)


In [ ]:
from langchain.chains import SequentialChain
first_prompt = PromptTemplate(
    input_variables=["topic"],
    template="Generate a short story idea about {topic}."
)
first_chain = LLMChain(
    llm=llm,
    prompt=first_prompt,
    output_key="story_idea"
)

second_prompt = PromptTemplate(
    input_variables=["story_idea", "genre"],
    template="Create a {genre} title for this story: {story_idea}"
)
second_chain = LLMChain(
    llm=llm,
    prompt=second_prompt,
    output_key="title"
)

sequential_chain = SequentialChain(
    chains=[first_chain, second_chain],
    input_variables=["topic", "genre"],
    output_variables=["story_idea", "title"],
    verbose=True
)
result = sequential_chain.invoke({
    "topic": "space exploration",
    "genre": "science fiction"
})
print(result)

## Router Chains

In [ ]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain_core.prompts import PromptTemplate

physics_template = """You are a physics expert. Answer this physics question:
{query}"""
math_template = """You are a math expert. Answer this math question:
{query}"""
history_template = """You are a history expert. Answer this history question:
{query}"""

physics_prompt = ChatPromptTemplate.from_template(physics_template)
math_prompt = ChatPromptTemplate.from_template(math_template)
history_prompt = ChatPromptTemplate.from_template(history_template)

destination_chains = {
    "physics": physics_prompt | llm,
    "math": math_prompt | llm,
    "history": history_prompt | llm
}

router_template = """Given a query, select the category it belongs to:
1. Physics: Questions about physics, forces, energy, etc.
2. Math: Questions about mathematics, calculations, etc.
3. History: Questions about historical events, periods, people, etc.

Query: {query}
Category:"""

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["query"]
)

router_chain = LLMRouterChain.from_llm(
    llm=llm,
    prompt=router_prompt
)

chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=chainllm,
    verbose=True
)

result = chain.invoke("What is the formula for kinetic energy?")

## Question Answering Chains

In [ ]:
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter

# Load document
loader = TextLoader("files/data.txt")
documents = loader.load()

# Split text
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

# Create vector store
vectorstore = Chroma.from_documents(documents=chunks, embedding=OpenAIEmbeddings())

# Create chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # Other options: "map_reduce", "refine", "map_rerank"
    retriever=vectorstore.as_retriever(),
    verbose=True
)

result = qa_chain.invoke("What is the main topic of the document?")

# With LCEL
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

retriever = vectorstore.as_retriever()

template = """Answer the question based on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

qa_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

response = qa_chain.invoke("What is the main topic of the document?")

# With source documents
from langchain.chains import RetrievalQAWithSourcesChain

qa_with_sources = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

response = qa_with_sources.invoke("What is the main topic of the document?")